In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float, PickleType
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import numpy as np

from Curvas import print_curve_data, import_calculated_monaco_ref_data

Base = declarative_base()

class CurveModel(Base):
    __tablename__ = 'INTECNUS_Curves'
    id = Column(Integer, primary_key=True)
    type = Column(String)
    coordinate = Column(String)
    date = Column(String)
    time = Column(String)
    depth = Column(Float)
    SSD = Column(Float)
    field_size = Column(Float)
    axis = Column(PickleType)  # para almacenar numpy arrays
    relative_dose = Column(PickleType)  # para almacenar numpy arrays
    machine = Column(String)
    particle = Column(Integer)
    energy = Column(Float)
    detector = Column(String)


C:\Users\roy.lapera\AppData\Local\Temp\ipykernel_5784\815585369.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
# Configura la conexión a la base de datos
engine = create_engine('sqlite:///Monaco_Reference_Data_Calculation.db')
Base.metadata.create_all(engine)

# Crea una sesión
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
class Curve:
    def __init__(self, type='', coordinate='', date=' ', time=' ', depth=0.0, SSD=0.0, 
                 field_size=0.0, axis=None, relative_dose=None, machine='', 
                 particle=0, energy=0.0, detector=''):
        self.type = type
        self.coordinate = coordinate
        self.date = date
        self.time = time
        self.depth = depth
        self.SSD = SSD
        self.field_size = field_size
        self.axis = axis if axis is not None else np.array([])
        self.relative_dose = relative_dose if relative_dose is not None else np.array([])
        self.machine = machine
        self.particle = particle
        self.energy = energy
        self.detector = detector

# Crear una lista de objetos Curve
monaco_path = 'P:/8 - Físicos Médicos/Roy/INTECNUS---RoyQA/MONACO'
curves = import_calculated_monaco_ref_data(monaco_path)

for curve in curves:
    print_curve_data(curve)

Equipo:      P, Fecha:       , Hora:        , Particula:   1, Energia:     15.0, DFS:         100.0, TC:          6.0, Coordenada:  X, Profundidad: 228.4, Detector:    N/A
Equipo:      P, Fecha:       , Hora:        , Particula:   1, Energia:     15.0, DFS:         100.0, TC:          6.0, Coordenada:  Y, Profundidad: 228.4, Detector:    N/A
Equipo:      P, Fecha:       , Hora:        , Particula:   1, Energia:     12.0, DFS:         100.0, TC:          6.0, Coordenada:  X, Profundidad: 227.69, Detector:    N/A
Equipo:      P, Fecha:       , Hora:        , Particula:   1, Energia:     12.0, DFS:         100.0, TC:          6.0, Coordenada:  Y, Profundidad: 227.69, Detector:    N/A
Equipo:      P, Fecha:       , Hora:        , Particula:   1, Energia:     9.0, DFS:         100.0, TC:          6.0, Coordenada:  X, Profundidad: 226.99, Detector:    N/A
Equipo:      P, Fecha:       , Hora:        , Particula:   1, Energia:     9.0, DFS:         100.0, TC:          6.0, Coordenada:  Y, Prof

In [9]:
for curve in curves:
    for attribute_name in curve.__dict__.keys():
        attribute_value = getattr(curve, attribute_name)
        print(f"Atributo: {attribute_name}, Tipo: {type(attribute_value).__name__}")


Atributo: type, Tipo: str
Atributo: date, Tipo: str
Atributo: time, Tipo: str
Atributo: detector, Tipo: str
Atributo: depth, Tipo: float
Atributo: field_size, Tipo: float
Atributo: SSD, Tipo: float
Atributo: energy, Tipo: float
Atributo: machine, Tipo: str
Atributo: particle, Tipo: int
Atributo: coordinate, Tipo: str
Atributo: axis, Tipo: ndarray
Atributo: relative_dose, Tipo: ndarray
Atributo: type, Tipo: str
Atributo: date, Tipo: str
Atributo: time, Tipo: str
Atributo: detector, Tipo: str
Atributo: depth, Tipo: float
Atributo: field_size, Tipo: float
Atributo: SSD, Tipo: float
Atributo: energy, Tipo: float
Atributo: machine, Tipo: str
Atributo: particle, Tipo: int
Atributo: coordinate, Tipo: str
Atributo: axis, Tipo: ndarray
Atributo: relative_dose, Tipo: ndarray
Atributo: type, Tipo: str
Atributo: date, Tipo: str
Atributo: time, Tipo: str
Atributo: detector, Tipo: str
Atributo: depth, Tipo: float
Atributo: field_size, Tipo: float
Atributo: SSD, Tipo: float
Atributo: energy, Tipo: fl

In [4]:
# Iterar sobre la lista de objetos Curve y guardarlos en la base de datos
for curve in curves:
    # Convertir depth a None si es 'N/A'
    depth = None if curve.depth == 'N/A' else float(curve.depth)
    
    curve_record = CurveModel(
        type=curve.type,
        coordinate=curve.coordinate,
        date=curve.date,
        time=curve.time,
        depth=curve.depth,
        SSD=curve.SSD,
        field_size=curve.field_size,
        axis=curve.axis,
        relative_dose=curve.relative_dose,
        machine=curve.machine,
        particle=curve.particle,
        energy=curve.energy,
        detector=curve.detector
    )
    session.add(curve_record)

# Confirmar (commit) los cambios en la base de datos
session.commit()

StatementError: (builtins.ValueError) could not convert string to float: 'N/A'
[SQL: INSERT INTO "INTECNUS_Curves" (type, coordinate, date, time, depth, "SSD", field_size, axis, relative_dose, machine, particle, energy, detector) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) RETURNING id]
[parameters: [{'time': '', 'relative_dose': array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00 ... (12477 characters truncated) ...      280.,  281.,  282.,  283.,  284.,  285.,  286.,  287.,  288.,
        289.,  290.,  291.,  292.,  293.,  294.,  295.,  296.]), 'detector': 'N/A'}, {'time': '', 'relative_dose': array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       1.0000e-01, 1.0000e-01, 0.0000e+00, 0.0000e+00 ... (10622 characters truncated) ... 6.,  237.,  238.,  239.,  240.,  241.,  242.,
        243.,  244.,  245.,  246.,  247.,  248.,  249.,  250.,  251.,
        252.]), 'detector': 'N/A'}, {'time': '', 'r